# Use CASH_OUT_Template Customized Notebook Template

In [6]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

import pandas as pd
import datetime as dt

In [7]:
#import seaborn as sns
import subprocess

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

In [8]:
from fosforml import register_model
import requests

In [9]:
# Import label encoder 
from sklearn import preprocessing 
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from math import sqrt
import numpy as np

In [10]:
table_name = 'ATM_TRANSACTION_MASTER_DATA'

sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [11]:
table_name = 'ATM_TRANSACTION_MASTER_DATA'

sf_df = my_session.sql("select * from {}".format(table_name))
df_original = sf_df.to_pandas()

In [13]:
df.columns

Index(['DATE', 'WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'BANK_NAME',
       'ATM_ID', 'DISPENSED_AMOUNT', 'TOTAL_TRANSACTION_COUNT',
       'DOWNTIME_IN_MINS', 'ATM_MAX_CAPACITY', 'MALE_POPULATION',
       'FEMALE_POPULATION', 'TOTAL_POPULATION', 'NO_OF_VETERANS',
       'FOREIGN_BORN', 'AVG_HOUSEHOLD_SIZE', 'LANDAREA_SQ_MILE',
       'POPULATION_RATIO', 'POPULATION_DENSITY', 'MEDIAN_HOUSE_INCOME',
       'LOCATION_TYPE', 'ATM_PLACEMENTS', 'ATM_ACCESSIBILITY',
       'ATM_MAINTAINENANCE', 'ATM__REPLENISHMENT'],
      dtype='object')

In [14]:
#to_drop = ['ATM_STATE_DETAILS_STATE','MALE_POPULATION','FEMALE_POPULATION','TOTAL_POPULATION','NO_OF_VETERANS','FOREIGN_BORN','AVG_HOUSEHOLD_SIZE','LANDAREA_SQ_MILE','POPULATION_RATIO','POPULATION_DENSITY','MEDIAN_HOUSE_INCOME']
to_drop = ['STATE','MALE_POPULATION','FEMALE_POPULATION','TOTAL_POPULATION','NO_OF_VETERANS','FOREIGN_BORN','AVG_HOUSEHOLD_SIZE','LANDAREA_SQ_MILE','POPULATION_RATIO','POPULATION_DENSITY','MEDIAN_HOUSE_INCOME']
df.drop(to_drop, inplace=True, axis=1)

In [ ]:
to_drop = ['ATM_STATE_DETAILS_STATE','MALE_POPULATION','FEMALE_POPULATION','TOTAL_POPULATION','NO_OF_VETERANS','FOREIGN_BORN','AVG_HOUSEHOLD_SIZE','LANDAREA_SQ_MILE','POPULATION_RATIO','POPULATION_DENSITY','MEDIAN_HOUSE_INCOME']
df_original.drop(to_drop, inplace=True, axis=1)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df['DATE'] = pd.to_datetime(df['DATE'])
df['HOUR'] = df['DATE'].dt.hour
df['DAYOFWEEK'] = df['DATE'].dt.dayofweek
df['QUARTER'] = df['DATE'].dt.quarter
df['MONTH'] = df['DATE'].dt.month
df['YEAR'] = df['DATE'].dt.year
df['DAYOFYEAR'] = df['DATE'].dt.dayofyear
df['DAYOFMONTH'] = df['DATE'].dt.day
df['DATE'] = pd.to_datetime(df['DATE']).dt.strftime("%Y-%m-%d %H:%M:%S.%f")

In [ ]:
df['WEEKDAY_FLAG'] = df['WEEKDAY_FLAG'].map({'Y': 1, 'N': 0})
df['HOLIDAY_FLAG'] = df['HOLIDAY_FLAG'].map({'Y': 1, 'N': 0})

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['STATE']= label_encoder.fit_transform(df['STATE'])

In [ ]:
# how to understand word labels. 
atm_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['ATM_ID']= atm_encoder.fit_transform(df['ATM_ID'])

In [ ]:
df.drop(['DATE','BANK_NAME','TOTAL_TRANSACTION_COUNT','DOWNTIME_IN_MINS','ATM_MAX_CAPACITY'], axis=1,inplace=True)

In [ ]:
df.columns

In [ ]:
features = ['WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'ATM_ID',
    'HOUR', 'DAYOFWEEK', 'QUARTER', 'MONTH', 'YEAR',
       'DAYOFYEAR', 'DAYOFMONTH']

In [ ]:
print (df.shape)
df = df[df['DISPENSED_AMOUNT'] > 0]
print (df.shape)

In [ ]:
X = df[features]
y = df['DISPENSED_AMOUNT']

In [ ]:
X.head()

In [ ]:
y

In [ ]:
# Split the data into training and test sets. (0.75, 0.25) split.
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)

In [ ]:
print(f'Total # of sample in whole dataset: {len(X)}')
print("*****"*10)
print(f'Total # of sample in train dataset: {len(X_train)}')
print(f'Shape of X_train: {X_train.shape}')
print("*****"*10)
print(f'Total # of sample in test dataset: {len(X_test)}')
print(f'Shape of X_test: {X_test.shape}')

# Linear Regression

In [ ]:
Lreg = LinearRegression()

In [ ]:
Lreg.fit(X_train,  y_train)

In [ ]:
predictions = Lreg.predict(X_test)

In [ ]:
print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

# Lasso Regression

In [ ]:
Lreg = Lasso()

In [ ]:
Lreg.fit(X_train,  y_train)
predictions = Lreg.predict(X_test)

print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

# Ridge Regression

In [ ]:
Lreg = Ridge()

In [ ]:
Lreg.fit(X_train,  y_train)
predictions = Lreg.predict(X_test)

print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

# K-Nearest Neighbour

In [ ]:
Lreg = KNeighborsRegressor()

In [ ]:
Lreg.fit(X_train,  y_train)

In [ ]:
predictions = Lreg.predict(X_test)

In [ ]:
print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

# Hyperparameter tunning

In [ ]:
for i in range(2,20,2):
    #Train Model and Predict  
    Lreg = KNeighborsRegressor(n_neighbors = i).fit(X_train,y_train)
    predictions = Lreg.predict(X_test)
    print ("--------------- Neighbour: ", i,"------------------------")
    print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
    print('Mean Squared Error:', mean_squared_error(y_test,predictions))
    print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
    print('r2_score:', r2_score(y_test,predictions))

In [ ]:
tree = KNeighborsRegressor(n_neighbors=10)

tree.fit(X_train,  y_train)
predictions = tree.predict(X_test)

print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

In [ ]:
y_pred = predictions
y_prediction = pd.Series(y_pred)

In [50]:
type(X_train), type(X_test), type(y_train), type(y_test), type(y_pred), type(y_prediction)

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.series.Series,
 pandas.core.series.Series,
 numpy.ndarray,
 pandas.core.series.Series)

In [51]:
## registering the model in Fosfor.
model_reg = register_model(tree,
               score, 
               name="ATM_DispenseAMT_KNN_10_Regression", 
               description="ATM Dispense Amount KNN_10 Regression",
               flavour=MLModelFlavours.sklearn,
               model_type="regression",
               #init_script="pip install snowflake-ml-python==1.0.11",
               init_script="pip install cloudpickle==2.2.1",
               y_true=y_test,
               y_pred=y_prediction,
               #prob=y_prob,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train.tolist(),
               y_test=y_test.tolist(),
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%
